# Linear-Feedback Shift Register (LFSR)

In an LFSR, the output from a standard shift register is fed back into its input in such a way as to cause the function to endlessly cycle through a sequence of patterns.


![LFSR](LFSR.png)

\\(
\begin{align}
    b[t]       = &\; s_0[t] \\
    s_j[t]     = &\; s_{j+1}[t-1] \\
    s_{m-1}[t] = &\; \oplus_{j=0}^{m-1} p_{m-j} \otimes s_{j}[t-1] \\
\end{align}
\\)

**Example**:
- LFSR length: 3
- feeedback polynomial: \\( x^3 + x + 1 \\)
- initial state: 0b111
- LFSR cycle:
|     state | output | feedback |
|:---------:|:------:|:--------:|
| 0b111 (7) |    1   |     0    |
| 0b011 (3) |    1   |     1    |
| 0b101 (5) |    1   |     0    |
| 0b010 (2) |    0   |     0    |
| 0b001 (1) |    1   |     1    |
| 0b100 (4) |    0   |     1    |
| 0b110 (6) |    0   |     1    |
| 0b111 (7) |    1   |     0    |

## Task 1: LFSR generator

> Define a **generator** that implements an LFSR. Given feeedback polynomial and an initial state, it generates an infinite stream of bit.

**Inputs**:
 - **Feedback Polynomial**: `list` of `int` representing the degrees of the non-zeros coefficients. Example: [12, 6, 4, 1, 0] represents the polynomial \\( x^{12}+x^6+x^4+x+1 \\).
 - **shift-register initial state** (optional, default all bits to 1): `int` or `list` of bits representing the LFSR initial state. Example: 0xA65 for 0b101001100101.
 
**Yield**:
- **Output bit**: `bool` representing the LFSR output bit

**Template**:
```python
def lfsr_generator(poly, state=None):
    ''' generator docstring '''
    
    # check inputs validity
    # define variables storing the internal state
    
    while True:
        # LFSR iteration:
        # - update state
        # - compute output
        yield output
```

### implentation with lists

In this first implementation both the shift register `state` and the feedback polynomial `poly` are `list` of `bool`. This is the most straightforward choice as it directly maps the LFSR block scheme.

| variable/<br>operation | &nbsp; &nbsp; &nbsp; &nbsp; definition &nbsp; &nbsp; &nbsp; &nbsp; | implementation |
|:-:|:-:|:--|
| feeedback <br> polynomial | &nbsp; \\( x^3 + x + 1 \\) | `poly = [True, True, False, True]` |
| initial state | 0b111 | `state = [True, True, True]` |
| state update | &nbsp; \\( s_j[t] = s_{j+1}[t-1] \\) | `state = state[1:] + [feedback]`
|
| output bit | &nbsp; \\( s_0[t] \\) | `output = state[0]` |
| feedback bit | &nbsp; \\( \oplus_{j=0}^{m-1} p_{m-j} \otimes s_{j}[t] \\) &nbsp; | `feedback = reduce(xor, compress(state[::-1], poly[1:]))` |

In [3]:
# xor and reduce from the built-in library operator and functools allows us to
# compute the parity of a bitstring in a nice and clean way
from operator import xor
from functools import reduce
from itertools import compress

def lfsr_generator(poly, state=None, verbose=False):
    '''
    Generator implementing a Linear Feedback Shift Register (LFSR)
    
    Parameters
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    state: int, optional (default=None),
        shift register initial state. If None, all bits of state are set to 1.
    verbose: bool, optional (default=False),
        If True, the internal state is printed at each iteration
    
    Return
    ------
    bool, LFSR output bit
    '''
    

    if verbose:
        print_lfsr = lambda b, s, f, l: print('{} ({})  {}  {}'.format(
            ''.join(str(int(b)) for b in state)[::-1], 
            int(''.join(str(int(b)) for b in state)[::-1], 2), 
            int(b), int(f),
        ))
    
    # compute LFSR length as the max degree of the feedback polynomial
    length = max(poly)
    
    # both poly and state stored as list of bool
    poly = [i in poly for i in range(length+1)]
    
    if state is None:
        # default value for state is all ones (True)
        state = [True for _ in range(length)]
    else:
        # convert integer to list of bool
        state = [bool(int(b)) for b in '{:0{}b}'.format(state, length)[::-1]]
        
    output = state[0]
    feedback = reduce(xor, compress(state[::-1], poly[1:]))
    
    if verbose: print(' state   b fb')
    if verbose: print_lfsr(output, state, feedback, length)
    
    # infinite loop
    while True:
        # update state by feeding with the feedback bit 
        state = state[1:] + [feedback]
        # get output from current state
        output = state[0]
        # update feedback bit 
        feedback = reduce(xor, compress(state[::-1], poly[1:]))
        
        if verbose: print_lfsr(output, state, feedback, length)
        
        yield output

In [4]:
# islice from the built-in itertools library allows you to perdorm a finite
# number of iteration on a iterator with infinite elements 
from itertools import islice

poly = [3, 1, 0] # feedback polynomial x^3 + x + 1
state = 0x7      # initial state
lfsr = lfsr_generator(poly, state, verbose=True) # lfsr definition

niter = 7        # number of iterations

# just iter over the LFSR generator to see the printed internal status
for b in islice(lfsr, niter):
    pass

 state   b fb
111 (7)  1  0
011 (3)  1  1
101 (5)  1  0
010 (2)  0  0
001 (1)  1  1
100 (4)  0  1
110 (6)  0  1
111 (7)  1  0


### implentation with integers

In this first implementation both the shift register `state` and the feedback polynomial `poly` are `int`. With this choice, bit-wise logical operation, as well as bit-shift, are easy to perform, while XOR of multiple bits or reversing the bit order are less straightforward.

| variable/<br>operation | &nbsp; &nbsp; &nbsp; &nbsp; definition &nbsp; &nbsp; &nbsp; &nbsp; | implementation |
|:-:|:-:|:--|
| feeedback <br> polynomial | &nbsp; \\( x^3 + x + 1 \\) | `poly = 0b1011`, `length = 3` |
| initial state | 0b111 | `state = 0b111` |
| state update | &nbsp; \\( s_j[t] = s_{j+1}[t-1] \\) | `statemask = (1 << length) - 1` <br> `state = ((state << 1) \| feedback) & statemask`
|
| output bit | &nbsp; \\( s_0[t] \\) | `outmask = 1 << (length-1)` <br> `output = bool(state & outmask)` |
| feedback bit | &nbsp; \\( \oplus_{j=0}^{m-1} p_{m-j} \otimes s_{j}[t] \\) &nbsp; | `feedback = parity(state & poly)` |

In [5]:
def lfsr_generator(poly, state=None, verbose=False):
    '''
    Generator implementing a Linear Feedback Shift Register (LFSR)
    
    Parameters
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    state: int, optional (default=None),
        shift register initial state. If None, all bits of state are set to 1.
    verbose: bool, optional (default=False),
        If True, the internal state is printed at each iteration
    
    Return
    ------
    bool, LFSR output bit
    '''
    
    # reverse takes an integer and returns the integer obtained by reversing
    # all bits (e.g., input: 11 = 0b1011 -> output: 13 = 0b1101)
    reverse = lambda x, n: int('{:0{n}b}'.format(x, n=n)[::-1], 2)
    # parity compute the parity bit ('0' even number of '1', '1' odd number
    # of '1') of an integer (e.g., input: 11 = 0b1011 -> output: True)
    parity = lambda x: bool(sum(int(b) for b in '{:b}'.format(x)) % 2)
    
    # print internal status in case verbose is set
    if verbose:
        print_lfsr = lambda b, s, f, l: print('{:0{}b} ({})  {}  {}'.format(
            reverse(s, l), l, reverse(s, l), int(b), int(f),))
    
    # compute LFSR length as the max degree of the feedback polynomial
    length = max(poly)
    # since state is kept as integer masks are needed
    outmask = 1 << (length-1)
    statemask = (1 << length) - 1
    
    # also poly stored as integer, ignoring first coefficient (always 1)
    poly = sum([2**p for p in poly]) >> 1
    
    # default value for state is all ones
    if state is None:
        state = statemask
    # reverse the bit order of state
    state = reverse(state, length)
    # get output from current state
    output = bool(state & outmask)
    # compute feedback bit as the parity bit (xor over each element)
    feedback = parity(state & poly)
    
    if verbose: print(' state   b fb')
    if verbose: print_lfsr(output, state, feedback, length)
    
    # infinite loop
    while True:
        # update state by feeding with the feedback bit 
        state = ((state << 1) | feedback) & statemask
        # get output from current state
        output = bool(state & outmask)
        # update feedback bit 
        feedback = parity(state & poly) 
        
        if verbose: print_lfsr(output, state, feedback, length)
        
        yield output

In [6]:
# islice from the built-in itertools library allows you to perdorm a finite
# number of iteration on a iterator with infinite elements 
from itertools import islice

poly = [3, 1, 0] # feedback polynomial x^3 + x + 1
state = 0x7      # initial state
lfsr = lfsr_generator(poly, state, verbose=True) # lfsr definition

niter = 7        # number of iterations

# just iter over the LFSR generator to see the printed internal status
for b in islice(lfsr, niter):
    pass

 state   b fb
111 (7)  1  0
011 (3)  1  1
101 (5)  1  0
010 (2)  0  0
001 (1)  1  1
100 (4)  0  1
110 (6)  0  1
111 (7)  1  0


In [7]:
# now we can use it to generate a random bit string

nbits = 100 # number of bits to generate
lfsr = lfsr_generator(poly, state) # lfsr definition

# iter over LFSR and store its output in a list
bits = [b for b in islice(lfsr_generator(poly, state), nbits)]

print(''.join([str(int(b)) for b in bits])) # each bool converted to '0'/'1'

1101001110100111010011101001110100111010011101001110100111010011101001110100111010011101001110100111


### implementation with bitstream

We can also try to implement an LFSR by using the classes `Bits` and `Bitarray` defined in the third-part package [`bitstring`](https://github.com/scott-griffiths/bitstring). Both Bits and Bitarray are containers for binary data but the former is immutable while the latter is mutable. Thanks to these two classes, you can manage bits as if they were either list of bool or int.

| variable/<br>operation | &nbsp; &nbsp; &nbsp; &nbsp; definition &nbsp; &nbsp; &nbsp; &nbsp; | implementation |
|:-:|:-:|:--|
| feeedback <br> polynomial | &nbsp; \\( x^3 + x + 1 \\) | `length = 3` <br> `Bits(uint=0b1101, length=length+1)` |
| initial state | 0b111 | `BitArray(uint=0b111, length=length)` |
| state update | &nbsp; \\( s_j[t] = s_{j+1}[t-1] \\) | `state.ror(1)` <br> `state[0] = feedback`
|
| output bit | &nbsp; \\( s_0[t] \\) | `output = state[-1]` |
| feedback bit | &nbsp; \\( \oplus_{j=0}^{m-1} p_{m-j} \otimes s_{j}[t] \\) &nbsp; | `feedback = reduce(xor, state[::-1] & poly[:-1])` |

In [8]:
# Bits is a immutable object that can be used for the feedback polynomial
# BitArray is a mutable object that can be used to deal with the LFSR state
from bitstring import Bits, BitArray

# xor and reduce from the built-in library operator and functools allows us to
# compute the parity of a bitstring in a nice and clean way
from operator import xor
from functools import reduce


def lfsr_generator(poly, state=None, verbose=False):
    '''
    Generator implementing a Linear Feedback Shift Register (LFSR)
    
    Parameters
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    state: int, optional (default=None),
        shift register initial state. If None, all bits of state are set to 1.
    verbose: bool, optional (default=False),
        If True, the internal state is printed at each iteration
    
    Return
    ------
    bool, LFSR output bit
    '''
    
    if verbose:
        print_lfsr = lambda b, s, f, l: print('{} ({})  {}  {}'.format(
            s.bin, s.uint, int(b), int(f),))
    
    length = max(poly)
    poly = Bits(uint=sum([2**p for p in poly]), length=length+1)
    
    if state is None:
        state = BitArray([True for i in range(length)])
    else:
        state = BitArray(uint=state, length=length)
    
    output = state[-1]
    feedback = reduce(xor, state[::-1] & poly[:-1])
    
    if verbose: print(' state   b fb')
    if verbose: print_lfsr(output, state, feedback, length)
        
    while True:
        # update state by feeding with the feedback bit 
        state.ror(1)
        state[0] = feedback
        # get output from current state
        output = state[-1]
        # update feedback bit 
        feedback = reduce(xor, state[::-1] & poly[:-1])
        
        if verbose: print_lfsr(output, state, feedback, length)
            
        yield output

In [9]:
from itertools import islice

poly = [3, 1, 0] # feedback polynomial x^3 + x + 1
state = 0x7      # initial state

niter = 7        # number of iterations

for b in islice(lfsr_generator(poly, state, verbose=True), niter):
    pass

 state   b fb
111 (7)  1  0
011 (3)  1  1
101 (5)  1  0
010 (2)  0  0
001 (1)  1  1
100 (4)  0  1
110 (6)  0  1
111 (7)  1  0


In [10]:
nbits = 100
lfsr = lfsr_generator(poly, state)

bits = [b for b in islice(lfsr, nbits)]

print(''.join([str(int(b)) for b in bits])) # each bool converted to '0'/'1'

1101001110100111010011101001110100111010011101001110100111010011101001110100111010011101001110100111


## Task 2: LFSR Iterator

> Transform the LFSR generator in an **iterator**, so that it is possible to access to the internal state.

**Inputs**:
 - **Feedback Polynomial**: `list` of `int` representing the degrees of the non-zeros coefficients. Example: [12, 6, 4, 1, 0] represents the polynomial \\(x^{12}+x^6+x^4+x+1\\).
 - **shift-register initial state** (optional, default all bits to 1): `int` or `list` of bits representing the LFSR initial state. Example: 0xA65 for 0b101001100101.
 
**Attributes**:
- `poly`: (`list` of `int`) list of the polynomial coefficients;
- `length`: (`int`) polynomial degree and length of the shift register;
- `state`: (`int`) LFSR state;
- `output`: (`bool`) output bit;
- `feedback`: (`bool`) feedback bit.
 
**Methods**:
- `__init__`: class constructor;
- `__iter__`: necessary to be an iterable;
- `__next__`: update LFSR state and returns output bit;
- `cycle`: returns a list of bool representing the full LFSR cycle ;
- `run_steps`: execute N LFSR steps and returns the corresponding output as list of bool (N is a input parameter, default N=1);
- `__str__`: return a string describing the LFSR class instance.


**Template**:
```python
class LFSR(object):
    ''' class docstring '''

    def __init__(self, poly, state=None):
        ''' constructor docstring '''
        ...
        self.poly = ... 
        self.length = ... 
        self.state = ... 
        self.output = ... 
        self.feedback = ...

    def __iter__(self): 
        return self

    def __next__(self): 
        ''' next docstring ''' 
        ... 
        return self.output

    def run_steps(self, N=1): 
        ''' run_steps docstring ''' 
        ... 
        return list_of_bool

    def cycle(self, state=None): 
        ''' cycle docstring ''' 
        ... 
        return list_of_bool
```

In [12]:
class LFSR(object):
    '''
    Class implementing a Linear Feedback Shift Register (LFSR)
    
    Attributes
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    state: int, bytes,
        state of the shift register. bytes are considered little endian.
    output: bool,
        last shift register output,
    length: int,
        length of the shift register as well as  maximum degree of the feedback
        polynomial.
    
    Methods
    -------
    run_steps(self, N=1)
        Execute multiple LFSR steps
    cycle(self, state=None)
        Execute a full cycle.
    
    '''
    
    def __init__(self, poly, state=None):
        '''
        Parameters
        ----------
        poly: list of int/bool, string,
            feedback polynomial expressed as list of integers corresponding to
            the degrees of the non-zero coefficients, or as list of 0/1
            coefficients in the form of list of bools or string.
        state: int, list of int/bool, string, optional (default=None)
            shift register initial state expressed as integer or as bit
            sequence in the form of list of bool or string.
            If None, state is set to all ones.
        '''
        if hasattr(poly, '__iter__') and len(poly) > 1:
            if all(int(p) in (0, 1) for p in poly):
                length = len(poly)
                _poly = sum([int(p) << i for i, p in enumerate(poly)])
            elif all(type(p) is int for p in poly):
                length = max(poly)
                _poly = sum([2**p for p in poly])
            else:
                raise TypeError('input type is not supported')
        else:
            raise TypeError('input type is not supported')

        self._statemask = (1 << length) - 1
        self._outmask = 1 << (length-1)
        
        if state is None:
            state = self._statemask
            
        self._poly = _poly >> 1
        self._len = length
        self.state = state
        self._feedback = self.parity(self._state & self._poly) 
        self._output = bool(self._state & self._outmask)

    @property
    def state(self):
        return self.reverse(self._state, len(self))
    
    @state.setter
    def state(self, state):
        if isinstance(state, int):
            _state = state
        elif isinstance(state, (bytes, bytearray)):
            _state = int.from_bytes(state, byteorder='little')
        elif hasattr(state, '__iter__')\
            and all(int(p) in (0, 1) for s in state):
            _state = sum([int(s) << i for i, s in enumerate(state)])
        else:
            raise TypeError('input type is not supported')
        self._state = self.reverse(_state & self._statemask, len(self))

    @property
    def len(self):
        return self._len
    
    @len.setter
    def len(self, val):
        raise AttributeError('Denied')
    
    @property
    def poly(self):
        poly_bin = '{:0{}b}'.format(self._poly, len(self))[::-1]
        poly = [0] + [i+1 for i, b in enumerate(poly_bin) if int(b) > 0]
        return list(reversed(poly))
    
    @poly.setter
    def poly(self, poly):
        raise AttributeError('Denied')

    @property
    def output(self):
        return self._output
    
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')

    @property
    def feedback(self):
        return self._feedback
    
    @feedback.setter
    def feedback(self, feedback):
        if not (type(feedback) is bool):
            raise TypeError('feedback must be bool')
        self._feedback = feedback
        
    def __str__(self):
        poly_str = ' + '.join(
            [('x^'+str(d) if d > 1 else 'x' if d == 1 else '1') 
             for d in self.poly])
        output = None if self.output is None else int(self.output)
        _str = 'poly: "{}", state: 0x{:0{}x}, output: {}'\
            .format(poly_str, self.state, (self.len+1)//4, output)
        return _str
        
    def __repr__(self):
        return 'LSFR({})'.format(self.__str__())
    
    def __len__(self):
        return self.len
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Execute a LFSR step and returns the output bit (bool)'''
        self._state = ((self._state << 1) | self._feedback) & self._statemask
        self._output = bool(self._state & self._outmask)
        self._feedback = self.parity(self._state & self._poly) 
        return self._output
    
    def run_steps(self, N=1):
        '''
        Execute multiple LFSR steps.
        
        Parameters
        ----------
        N: int, optional (default=1)
            number of steps to execute.
        
        Output
        ------
        list of bool (len=N),
            LFSR output bit stream.
        '''
        return [next(self) for _ in range(N)]
#         return [b for b in islice(self, N)]
   
    def cycle(self, state=None):
        '''
        Execute a full LFSR cycle (LFSR.len steps).
        
        Parameters
        ----------
        state: int or list of int or bools, optional (default=None)
            shift register state. If None, state is kept as is.
        
        Output
        ------
        list of bool (len=2**myLFSR.len - 1),
            LFSR output bit stream.
        '''
        if state is not None:
            self.state = state
        return self.run_steps(N=int(2**self.len) - 1)
    
    @staticmethod
    def parity(x):
        '''Returns the parity bit (bool) of the input `x` (int)'''
        return bool(sum(int(b) for b in '{:b}'.format(x)) % 2)
    
    @staticmethod
    def reverse(x, n):
        '''Reverses the order of the `n` (int) bits in the input `x` (int)'''
        return int('{:0{}b}'.format(x, n)[::-1], 2)


In [13]:
poly = [3, 1, 0] # feedback polynomial x^3 + x + 1
state = 0x7      # initial state

niter = 7        # number of iterations


print_lfsr = lambda lfsr: print('{:0{}b} ({:d})  {:d}  {:d}'.format(
    lfsr.state, len(lfsr), lfsr.state, int(lfsr.output), int(lfsr.feedback)))

# create and instance of the LFSR
lfsr = LFSR(poly, state)

print(' state   b fb')
print_lfsr(lfsr)
for b in islice(lfsr, niter):
    print_lfsr(lfsr)

 state   b fb
111 (7)  1  0
011 (3)  1  1
101 (5)  1  0
010 (2)  0  0
001 (1)  1  1
100 (4)  0  1
110 (6)  0  1
111 (7)  1  0


## Task 3: Comparison with pylfsr

> Compare the LFSR iterator with the one provided by the pylfsr Python package.


[`pylfsr`](https://github.com/Nikeshbajaj/Linear_Feedback_Shift_Register) implements a class `LFSR` similar to the one implemented above. Here are some of the methods available in the that class:
- `next()`: run one cycle on LFSR with given feedback polynomial and update the state, feedback bit, and output bit.
- `runKCycle(k)`: run k cycles and update all the Parameters.
- `runFullCycle()`: run full cycle ( = 2^M-1, where M is the poly degree)
- `info()`: display the information about LFSR


In [15]:
from pylfsr import LFSR as pyLFSR

In [16]:
# poly, state = [3, 1, 0], 0x7
# poly, state = [4, 1, 0], 0xf
# poly, state = [5, 2, 0], 0x1f
# poly, state = [6, 1, 0], 0x3f
# poly, state = [7, 1, 0], 0x7f
# poly, state = [8, 4, 3, 2, 0], 0xff
# poly, state = [9, 4, 0], 0x1ff
# poly, state = [10, 3, 0], 0x3ff
poly, state = [11, 2, 0], 0x7ff

print('poly: {}, state: 0x{:0{}x}'.format(poly, state, (max(poly)+1)//4))

print('\n**** myLFSR ****')
mylfsr = LFSR(poly, state)
print(mylfsr)
mycycle = mylfsr.cycle()
print(''.join([str(int(b)) for b in mycycle]))


print('\n**** pyLFSR ****')
m = max(poly)
pylfsr = pyLFSR(
    fpoly=[p for p in poly if p > 0], 
    initstate=[int(b) for b in '{:0{}b}'.format(state, max(poly))],
    verbose=False,
)
pylfsr.info()
pycycle = pylfsr.runFullCycle()
print(('{}'*len(pycycle)).format(*pycycle))


print('\n**** Do both LFSRs give the same bit stream? ****')
print(all(int(my) == int(py) for my, py in zip(mycycle, pycycle)))

poly: [11, 2, 0], state: 0x7ff

**** myLFSR ****
poly: "x^11 + x^2 + 1", state: 0x7ff, output: 1
111111111100110011001011010010111000101110010100011010001010010000010000101010000001000101010111110111011001110010010110000101110000001001111111101001100110111100001110011111100101100110100011110001010011010111010010001101111010110110010001110000101001111110111100110001100001111100000011001010101101000100011101011111001000110010111110000100110000001011010101000111011101011000110111000001110010101001111011101001101100010110110101110001110110000011011010101101101110110110110001110001111100101001100001000011111101010011001000101101000001001000000010111111111011001100111000011110010101100101110110100011011011111000111001101011000011101101010011100010000110101111110001001100101000011110111111001110011000011010010101101111011100011000110101101011011100010010011111010000110010000001111010101001101110111100011011001010010010111010000100111010100001101110101001001110111101001110011101001111001000011

## Task 4: Berlekamp-Massey Algorithm

> Define a function that implements the Berlekamp-Massey Algorithm which finds the shortest LFSR that can generate the input bit stream.

**Pseudocode**:
> **Input** \\( b = [b_0, b_1, \dots, b_N]\\) <br>
> \\( P(x) \leftarrow 1, \quad m \leftarrow 0 \\) <br>
> \\( Q(x) \leftarrow 1, \quad r \leftarrow 1 \\) <br>
> **for** \\( \tau = 0, 1, \dots, N-1 \\) <br>
> \\( \qquad d \leftarrow \oplus_{j=0}^{m} p_j \otimes b[\tau-j] \\) <br>
> \\( \qquad \\) **if** \\( d = 1 \\) **then** <br>
> \\( \qquad \qquad \\) **if** \\( 2m \le \tau \\) **then** <br>
> \\( \qquad \qquad \qquad R(x) \leftarrow P(x) \\) <br>
> \\( \qquad \qquad \qquad P(x) \leftarrow P(x) + Q(x)x^r \\) <br>
> \\( \qquad \qquad \qquad Q(x) \leftarrow R(x) \\) <br>
> \\( \qquad \qquad \qquad m \leftarrow \tau + 1 - m \\) <br>
> \\( \qquad \qquad \qquad r \leftarrow 0 \\) <br>
> \\( \qquad \qquad \\) **else** <br>
> \\( \qquad \qquad \qquad P(x) \leftarrow P(x) + Q(x)x^r \\) <br>
> \\( \qquad \qquad \\) **endif** <br>
> \\( \qquad \\) **endif** <br>
> \\( \qquad r \leftarrow r + 1 \\) <br>
> **endfor** <br>
> **Output** \\( P(x), m \\) <br>

**Template**:
```python
def berlekamp_massey(b):
    ''' function docstring '''
    # algorithm implementation
    return poly, m
```

**Example**:

Input: \\( b = [1, 0, 1, 0, 0, 1, 1, 1] \\)

| \\( \tau \\) | \\( b_\tau \\) | \\( d \\) | \\( P(x) \\) | \\( m \\) | \\( Q(x) \\) | \\( r \\) |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| - | - | - | \\( 1           \\) | 0 | \\( 1       \\) | 1 |
| 0 | 1 | 1 | \\( 1 + x       \\) | 1 | \\( 1       \\) | 1 |
| 1 | 0 | 1 | \\( 1           \\) | 1 | \\( 1       \\) | 2 |
| 2 | 1 | 1 | \\( 1 + x^2     \\) | 2 | \\( 1       \\) | 1 |
| 3 | 0 | 0 | \\( 1 + x^2     \\) | 2 | \\( 1       \\) | 2 |
| 4 | 0 | 1 | \\( 1           \\) | 3 | \\( 1 + x^2 \\) | 1 |
| 5 | 1 | 1 | \\( 1 + x + x^3 \\) | 3 | \\( 1 + x^2 \\) | 2 |
| 6 | 1 | 0 | \\( 1 + x + x^3 \\) | 3 | \\( 1 + x^2 \\) | 3 |
| 7 | 1 | 0 | \\( 1 + x + x^3 \\) | 3 | \\( 1 + x^2 \\) | 4 |

Output: \\( P(x) = 1 + x + x^3 \\), \\( m = 3 \\)

In [17]:
def parity(x):
    '''Returns the parity bit (bool) of the input `x` (int)'''
    return bool(sum(int(b) for b in '{:b}'.format(x)) % 2)

def reverse(x, n):
    '''Reverses the order of the `n` (int) bits in the input `x` (int)'''
    return int('{:0{}b}'.format(x, n)[::-1], 2)

def discrepancy(P, m, b, t):
    '''
    Compute discrepancy (bool) between the Polynomial `P` (int) with degree `m`
    (int) and the bit sequence `b` (list/string of 0/1) at time `t` (int)
    '''
    # select bits b_t, b_{t-1}, ..., b_{t-j} in reverse order
    bbits = b[t:t-m-1 if t > m else None:-1]
    # transform bits in integer
    bint = sum([1 << i for i, bit in enumerate(bbits) if bool(int(bit))])
    # compute discrepancy 
    d = parity(P & bint)
    return d

def berlekamp_massey(b, verbose=False):
    '''
    Find the shortest LFSR for a given binary sequence.
    
    Parameters
    ----------
    b: list/string of 0/1,
        stream of bits.
        
    Return
    ------
    list of int,
        linear feedback polynomial expressed as list of the exponents related
        to the non-zero coefficients
    '''
    
    # variables initialization
    P, m = 0x1, 0
    Q, r = 0x1, 1
    
    if verbose:
        from math import log2
        lenP = 1 + int(log2(len(b)+1))
        print('t b d {:>{}} m {:>{}} r'.format('poly', lenP, 'Q', lenP))
        print('- - - {:{}} 0 {:{}} 1'.format(P, lenP, Q, lenP))
    
    for t in range(len(b)):
        # compute discrepancy
        d = discrepancy(P, m, b, t)
        if d:
            if 2*m <=t: # A                
                P, Q = P^(Q<<r), P
                m, r = t+1-m, 0
            else: # B
                P = P^(Q<<r)
        r += 1
        
        if verbose:
            print(t, int(b[t]), int(d), '{:{}b}'.format(P, lenP), m, 
                  '{:{}b}'.format(Q, lenP), r)
            
    # convert poly from integer to list of degree of non-zero coefficients
    poly = [i for i, p in enumerate('{:b}'.format(P)[::-1]) if bool(int(p))]
    return list(reversed(sorted(poly)))

In [18]:
b = '1010011101'
poly = berlekamp_massey(b, verbose=True)
print(poly)

t b d poly m    Q r
- - -    1 0    1 1
0 1 1   11 1    1 1
1 0 1    1 1    1 2
2 1 1  101 2    1 1
3 0 0  101 2    1 2
4 0 1    1 3  101 1
5 1 1 1011 3  101 2
6 1 0 1011 3  101 3
7 1 0 1011 3  101 4
8 0 0 1011 3  101 5
9 1 0 1011 3  101 6
[3, 1, 0]


In [ ]:
# poly, state = [3, 1, 0], 0x5
# poly, state = [4, 1, 0], 0x1
# poly, state = [5, 2, 0], 0x1
# poly, state = [6, 1, 0], 0x1
# poly, state = [7, 1, 0], 0x1
# poly, state = [8, 4, 3, 2, 0], 0x1
# poly, state = [9, 4, 0], 0x1
# poly, state = [10, 3, 0], 0x1
poly, state = [11, 2, 0], 0x1

lfsr = LFSR(poly, state)
print(lfsr)
# bitstream = lfsr.run_steps(N=2**len(lfsr))
bitstream = lfsr.cycle()
print(''.join([str(int(b)) for b in bitstream]))


bm_poly = berlekamp_massey(bitstream)
print(bm_poly)